# Ride-sharing analytics
As a Data Scientist working for Zuber, a new ride-sharing company that's launching in Chicago, you have been tasked with finding patterns in the available information. You are to understand passenger preferences and the impact of external factors on rides. You'll study a database, analyze data from competitors, and test a hypothesis about the impact of weather on ride frequency.

# Data description
A database with info on taxi rides in Chicago: 

`neighborhoods` table: data on city neighborhoods
- name: name of the neighborhood
- neighborhood_id: neighborhood code 

`cabs` table: data on taxis
- cab_id: vehicle code
- vehicle_id: the vehicle's technical ID
- company_name: the company that owns the vehicle

`trips` table: data on rides
- trip_id: ride code
- cab_id: code of the vehicle operating the ride
- start_ts: date and time of the beginning of the ride (time rounded to the hour)
- end_ts: date and time of the end of the ride (time rounded to the hour)
- duration_seconds: ride duration in seconds
- distance_miles: ride distance in miles
- pickup_location_id: pickup neighborhood code
- dropoff_location_id: dropoff neighborhood code

`weather_records` table: data on weather
- record_id: weather record code
- ts: record date and time (time rounded to the hour)
- temperature: temperature when the record was taken
- description: brief description of weather conditions, e.g. "light rain" or "scattered clouds"

# Objectives

The objective of this project is to:
- Write a code to parse the data on weather in Chicago in November 2017 from the [website](https://code.s3.yandex.net/data-analyst-eng/chicago_weather_2017.html)
- Understand passenger preferences and the impact of external factors on rides.
- Test a hypothesis about the impact of weather on ride frequency.

In [ ]:
# !pip install psycopg2

import psycopg2
import pandas as pd

conn = psycopg2.connect(
    host = 'localhost',
    database = 'postgres', 
    user = 'postgres', 
    password = 'chuksoo',  
    port = '5432')
print('Connection to database is successfully')

In [2]:
# function to read from database
def read(conn, read_):
    print('Read')
    cursor = conn.cursor()
    cursor.execute(read_)
    for row in cursor:
        print(f'row = {row}')
    print()
    
# function create in postgre database     
def create(conn, create_):
    cursor = conn.cursor() # create cursor object
    cursor.execute(create_) # execute query
    conn.commit() # commit query to database
    print('Table have been created successfull!!!')
    #read(conn)
    
# function insert in postgre database     
def insert(conn, insert_):
    cursor = conn.cursor()
    cursor.execute(insert_)
    conn.commit()
    print('Records have been successfully inserted!!!')
    #read(conn)
    
# function to update 
def update(conn, update_):
    print('Update')
    cursor = conn.cursor()
    cursor.execute(update_)
    conn.commit()
    #read(conn)
    
# function to delete in postgre database
def delete(conn, delete_):
    print('Delete')
    cursor = conn.cursor()
    cursor.execute(delete_)
    conn.commit()
    #read(conn)

# close the cursor and connection to the server 
def close():
    cursor.close()
    conn.close()
    
# function to create pandas dataframe
def create_pandas_df(sql_query, database=conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [3]:
# create table coaches
create_ = '''DROP TABLE IF EXISTS coaches;
            CREATE TABLE coaches(
                CoachID INT NULL,
                CoachName VARCHAR(50) NULL,
                CoachType VARCHAR(50) NULL,
                StartDate DATE NULL)'''
create(conn, create_)

# insert into table coaches
insert_ = '''INSERT INTO coaches (CoachID, CoachName, CoachType, StartDate)
            VALUES 
                (1, 'Mike Ramsey', 'Wide receiver', '1/1/2020'),
                (2,'Kliff Kingsbury', 'Quarterback Coach','1/1/2019'),
                (3,'Arthur Smith', 'Wide receiver','1/1/2021'),
                (4,'John Harbaugh','Offense','1/1/2008'),
                (5,'Kevin Stefanski','Offense','1/1/2020'),
                (6,'Vic Fangio','Defense','1/1/2019'),
                (7,'David Culley', 'Wide receiver','1/1/2021'),
                (8,'Frank Reich','Quarterback','1/1/2018'),
                (9,'Brandon Staley','Defense','1/1/2021'),
                (10,'Mike Zimmer', 'Running Back','1/1/2017');'''
insert(conn, insert_)

Table have been created successfull!!!
Records have been successfully inserted!!!


In [4]:
read_ = '''SELECT
                *
            FROM
                coaches
        '''
#read(conn, read_)

# output postgre query in pandas dataframe
coaches = create_pandas_df(read_, database=conn)
coaches.head(10)

,coachid,coachname,coachtype,startdate
0,1,Mike Ramsey,Wide receiver,2020-01-01
1,2,Kliff Kingsbury,Quarterback Coach,2019-01-01
2,3,Arthur Smith,Wide receiver,2021-01-01
3,4,John Harbaugh,Offense,2008-01-01
4,5,Kevin Stefanski,Offense,2020-01-01
5,6,Vic Fangio,Defense,2019-01-01
6,7,David Culley,Wide receiver,2021-01-01
7,8,Frank Reich,Quarterback,2018-01-01
8,9,Brandon Staley,Defense,2021-01-01
9,10,Mike Zimmer,Running Back,2017-01-01


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

URL = 'https://code.s3.yandex.net/data-analyst-eng/chicago_weather_2017.html'
req = requests.get(URL) 
soup = BeautifulSoup(req.text, 'lxml')

# extract the table of information for Chicago Weather Record using BeautifulSoup
weather = soup.find('table', attrs={'id': 'weather_records'})

# get names of columns for the datarame
heading_table = []
for row in weather.find_all('th'): # column names are inside <th>
    heading_table.append(row.text)
#print(heading_table)
 
# get content of the dataframe
content = []
for row in weather.find_all('tr'): # loop through <tr> tag
    if not row.find_all('th'): # ignore first row of table
        content.append([element.text for element in row.find_all('td')]) # cell content are inside <td> tags
#print(content)

# parse into a dataframe
weather_records = pd.DataFrame(content, columns=heading_table)
print(weather_records)

**Exploratory data analysis**
1. Print the *company_name* field. Find the number of taxi rides for each taxi company for November 15-16, 2017, name the resulting field *trips_amount*, and print it, too. Sort the results by the *trips_amount* field in descending order.

In [ ]:
%sql
SELECT
    cabs.company_name, COUNT(trips.trip_id) AS trips_amount
FROM
    cabs
JOIN trips
    ON   cabs.cab_id = trips.cab_id
WHERE 
    trips.start_ts::date BETWEEN '2017-11-15' AND '2017-11-16'
GROUP BY
   cabs.company_name
ORDER BY
    trips_amount DESC

2. Find the number of rides for every taxi company whose name contains the words "Yellow" or "Blue" for November 1-7, 2017. Name the resulting variable *trips_amount*. Group the results by the *company_name* field.

In [ ]:
%sql
SELECT
    cabs.company_name, COUNT(trips.trip_id) AS trips_amount
FROM
    cabs
JOIN trips
    ON cabs.cab_id = trips.cab_id
WHERE
    cabs.company_name LIKE '%Yellow%' 
    AND trips.start_ts::date BETWEEN '2017-11-01' AND '2017-11-07'
GROUP BY
    cabs.company_name
UNION 
SELECT
    cabs.company_name, COUNT(trips.trip_id) AS trips_amount
FROM
    cabs
JOIN trips
    ON cabs.cab_id = trips.cab_id
WHERE
    cabs.company_name LIKE '%Blue%'
    AND trips.start_ts::date BETWEEN '2017-11-01' AND '2017-11-07'
GROUP BY
    cabs.company_name

3. For November 1-7, 2017, the most popular taxi companies were Flash Cab and Taxi Affiliation Services. Find the number of rides for these two companies and name the resulting variable *trips_amount*. Join the rides for all other companies in the group "Other." Group the data by taxi company names. Name the field with taxi company names *company*. Sort the result in descending order by *trips_amount*.

In [ ]:
%sql
SELECT 
    CASE 
        WHEN company_name = 'Flash Cab' THEN 'Flash Cab' 
        WHEN company_name = 'Taxi Affiliation Services' THEN 'Taxi Affiliation Services' 
        ELSE 'Other' 
    END AS company,
    COUNT(trips.trip_id) AS trips_amount
FROM
    cabs
JOIN trips
    ON cabs.cab_id = trips.cab_id
WHERE
    trips.start_ts::date BETWEEN '2017-11-01' AND '2017-11-07'
GROUP BY
    company 
ORDER BY
    trips_amount DESC

**Test the hypothesis that the duration of rides from the the Loop to O'Hare International Airport changes on rainy Saturdays**

4. Retrieve the identifiers of the O'Hare and Loop neighborhoods from the neighborhoods table.

In [ ]:
SELECT
    neighborhood_id, name
FROM
    neighborhoods
WHERE
    name LIKE '%Hare' OR name LIKE 'Loop'

5. For each hour, retrieve the weather condition records from the weather_records table. Using the CASE operator, break all hours into two groups: `Bad` if the description field contains the words `rain` or `storm`, and `Good` for others. Name the resulting field *weather_conditions*. The final table must include two fields: date and hour (ts) and *weather_conditions*.

In [ ]:
SELECT
    ts,
    CASE
        WHEN description LIKE '%rain%' OR description LIKE '%storm%' THEN 'Bad'
        ELSE 'Good'
    END AS weather_conditions
FROM
    weather_records

6. Retrieve from the trips table all the rides that started in the Loop (*pickup_location_id*: 50) on a Saturday and ended at O'Hare (*dropoff_location_id*: 63). Get the weather conditions for each ride. Use the method you applied in the previous task. Also, retrieve the duration of each ride. Ignore rides for which data on weather conditions is not available.

The table columns should be in the following order:
- start_ts
- weather_conditions
- duration_seconds

Sort by *trip_id*.

In [ ]:
SELECT
    trips.start_ts, 
    CASE
        WHEN weather_records.description LIKE '%rain%' OR weather_records.description LIKE '%storm%' THEN 'Bad'
        ELSE 'Good'
    END AS weather_conditions,
    trips.duration_seconds
FROM
    trips 
JOIN weather_records 
ON   trips.start_ts = weather_records.ts
WHERE EXTRACT(DOW FROM trips.start_ts) = 6 AND
        trips.pickup_location_id = 50 AND trips.dropoff_location_id = 63
ORDER BY
    trips.trip_id